# This is a trial-and-error script

In [2]:
import os
import pandas as pd

In [8]:
# Load Fulltext Data
df_claims = pd.read_csv('/Users/juergenthiesen/Documents/Patentsview/Cleantech Concepts/g_claims_2000.tsv', sep='\t', encoding='utf-8')
df_brf_summary = pd.read_csv('/Users/juergenthiesen/Documents/Patentsview/Cleantech Concepts/g_brf_sum_text_2000.tsv', sep='\t', encoding='utf-8')
df_desc = pd.read_csv('/Users/juergenthiesen/Documents/Patentsview/Cleantech Concepts/g_detail_desc_text_2000.tsv', sep='\t', encoding='utf-8')
df_claims['patent_id'] = df_claims['patent_id'].astype(str)
df_brf_summary['patent_id'] = df_brf_summary['patent_id'].astype(str)
df_desc['patent_id'] = df_desc['patent_id'].astype(str)

In [10]:
# Sort df_claims by patent_id and then by claim_sequence
df_claims.sort_values(by=['patent_id', 'claim_sequence'], inplace=True)
df_brf_summary.sort_values(by=['patent_id'], inplace=True)
df_desc.sort_values(by=['patent_id'], inplace=True)

In [5]:
# Load Patent Data
df_patent_id = pd.read_csv('/Users/juergenthiesen/Documents/Patentsview/Cleantech Concepts/g_patent_ids_rel_on_science_cleantech.csv')
df_patent_id['patent_id'] = df_patent_id['patent_id'].astype(str)

In [11]:
# Match df_patent_id and df_claims on patent_id
df_patent_claims = df_claims.merge(df_patent_id, on='patent_id', how='right', validate='many_to_one')
df_patent_brf_summary = df_brf_summary.merge(df_patent_id, on='patent_id', how='right', validate='many_to_one')
df_patent_desc = df_desc.merge(df_patent_id, on='patent_id', how='right', validate='many_to_one')

In [5]:
# Group by patent_id
df_patent_claims_grouped = df_patent_claims.groupby('patent_id').agg({
    'claim_sequence': list,
    'claim_text': list,
    'dependent': list,
    'claim_number': list,
    'exemplary': list,
}).reset_index()

In [6]:
# Create dictionary of claims
df_patent_claims_grouped['claims'] = df_patent_claims_grouped.apply(lambda row: {i+1: {'claim_sequence': claim_sequence, 'claim_text': claim_text, 'dependent': dependent, 'claim_number': claim_number, 'exemplary': exemplary} for i, (claim_sequence, claim_text, dependent, claim_number, exemplary) in enumerate(zip(row['claim_sequence'], row['claim_text'], row['dependent'], row['claim_number'], row['exemplary']))}, axis=1)

In [15]:
# Drop original columns
df_patent_claims_grouped.drop(['claim_sequence', 'claim_text', 'dependent', 'claim_number', 'exemplary'], axis=1, inplace=True)

In [31]:
# Save to json
df_patent_claims_grouped.to_json('/Users/juergenthiesen/Documents/Patentsview/Cleantech Concepts/g_claims_2000.json', orient='records')
df_patent_brf_summary.to_json('/Users/juergenthiesen/Documents/Patentsview/Cleantech Concepts/g_brf_sum_text_2000.json', orient='records')
df_patent_desc_grouped = df_patent_desc.groupby('patent_id').apply(lambda x: {'description_text': x['description_text'].iloc[0], 'description_length': x['description_length'].iloc[0]}).reset_index(name='description')